<a href="https://colab.research.google.com/github/hseyeon1006/ESAA_OB/blob/main/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B82_(1)_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import random
import logging
from IPython.display import display, HTML
import torch

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from scipy.special import softmax
from sklearn.model_selection import StratifiedKFold
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ESAA/2학기/프로젝트/train_data.csv')
test = pd.read_csv('/content/drive/MyDrive/ESAA/2학기/프로젝트/test_data.csv')
submission = pd.read_csv('/content/drive/MyDrive/ESAA/2학기/프로젝트/sample_submission.csv')

In [ ]:
test = load_dataset('csv',data_files={"test":"/content/drive/MyDrive/ESAA/2학기/프로젝트/test_data.csv"})
submission = pd.read_csv('/content/drive/MyDrive/ESAA/2학기/프로젝트/sample_submission.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # for faster training, but not deterministic

In [ ]:
set_seeds(seed=20)
model_checkpoint = "klue/roberta-large"
batch_size = 32
task = "nli"

In [ ]:
# 위키트리 같은 곳을 통한 단어를 가지고 있는 곳
datasets = load_dataset("klue", task)

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
# GLUE (General Language Understanding Evaluation) 벤치마크의 하나인 QNLI (Question-answering Natural Language Inference) 데이터셋에 대한 평가 지표를 로드하는 역할
metric = load_metric("glue", "qnli") # 다양한 평가지표를 불러올 수 있음

In [ ]:
sentence1_key, sentence2_key = ("premise", "hypothesis")

def preprocess_function(examples):
    return tokenizer(                #입력 문자열을 token id로 변환(encoding), token id를 다시 문자열로 변환(decoding)의 기능
        examples[sentence1_key],
        examples[sentence2_key],
        truncation=True,             # True 인 경우 모델이 입력으로 받을 수 있는 최대의 길이에 맞춰서 Truncation
        return_token_type_ids=False, # token type id를 제공할지 여부. 기본적으로 제공함
    )

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 24998
    })
    validation: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 3000
    })
})

In [ ]:
! pip install -U accelerate
! pip install -U transformers

ERROR: Operation cancelled by user


In [ ]:
import accelerate

accelerate.__version__

'0.24.1'

In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.35.2', '0.24.1')

In [ ]:
metric_name = "accuracy"

# label을 숫자로 설정
map_dict = {'entailment': 0, 'neutral': 1, 'contradiction': 2}

# trainer 정의 전 모델 정의
args = TrainingArguments(
    "test-nli",
    evaluation_strategy="epoch",
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
encoded_datasets = datasets.map(preprocess_function, batched=True)
encoded_test = test['test'].remove_columns("label").map(preprocess_function, batched=True, remove_columns = ['premise', 'hypothesis'])

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Map:   0%|          | 0/24998 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1666 [00:00<?, ? examples/s]

In [ ]:
skf = StratifiedKFold(n_splits=10, random_state=2022, shuffle=True)
tot_train = concatenate_datasets([encoded_datasets['train'],encoded_datasets['validation']])
tot_train_pd = pd.DataFrame(tot_train)

In [ ]:
for n_fold, (_,v_idx) in enumerate(skf.split(tot_train_pd, tot_train_pd['label'])):
    tot_train_pd.loc[v_idx, 'fold']  = n_fold

In [ ]:
for i in range(0,10):
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)
    trainer = Trainer(
      model,
      args,
      train_dataset=Dataset.from_pandas(tot_train_pd[tot_train_pd['fold']!=i]),
      eval_dataset=Dataset.from_pandas(tot_train_pd[tot_train_pd['fold']==i]),
      tokenizer=tokenizer,
      compute_metrics=compute_metrics,
    )
    trainer.train()
    outputs = trainer.predict(encoded_test)
    prob = softmax(outputs[0],axis=1)

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.486500,0.319300,0.895000
2,0.180000,0.354484,0.897143
3,0.108200,0.486129,0.905000
4,0.045200,0.515087,0.906786
5,0.026600,0.613406,0.907857


FileNotFoundError: ignored

In [ ]:
# Transform the test outputs for the submission.
df_sub = pd.DataFrame({"index":encoded_test['index'], "label":np.argmax(outputs.predictions, axis=-1)})
df_sub['label'] = df_sub['label'].apply(lambda x: {v:k for k,v in map_dict.items()}[x])
df_sub.to_csv("/content/drive/MyDrive/ESAA/2학기/프로젝트/submission.csv", index=False)

https://dacon.io/competitions/official/235875/codeshare/4597?page=2&dtype=view

public점수 : 0.881
private점수 : 0.8799519808

In [ ]:
import numpy as np
import pandas as pd

from transformers import pipeline, AutoTokenizer
from tqdm import tqdm

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")
model = pipeline("text-classification", model="Huffon/klue-roberta-base-nli", return_all_scores=False)

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/ESAA/2학기/프로젝트/test_data.csv")
X_test = df_test["premise"] + " " + tokenizer.sep_token + " " + df_test["hypothesis"]

In [ ]:
y_preds = []
for i in tqdm(range(X_test.shape[0])):
    y_pred = model(X_test[i])[0]["label"].lower()
    y_preds.append(y_pred)

y_preds = np.array(y_preds)

100%|██████████| 1666/1666 [04:52<00:00,  5.69it/s]


In [ ]:
df_test["label"] = y_preds
df_submission = df_test.loc[:, ["index", "label"]]
df_submission.to_csv("/content/drive/MyDrive/ESAA/2학기/프로젝트/results.csv", index=False)

https://dacon.io/competitions/official/235875/codeshare/4424?page=1&dtype=view

public점수 : 0.83
private점수 : 0.824729892